## Dependencies

In [147]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase
from scipy.ndimage import rotate
import os
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3  # Assuming SQL connection for database operations
import matplotlib.font_manager as font_manager

# 
font_path = 'C:/Windows/Fonts/Exo 2.tff'

### DEFINE TEMP FOLDER - MAKE SURE IT IS NOT OVERWRITTEN BELOW
TEMP_FOLDER = '../../NCAA/TEMP/'

DATA_FOLDER = '../../NCAA/data/'



## OVERALL - ALL D1 Players

In [148]:
# Load roster data
roster_df = pd.read_csv(f'{DATA_FOLDER}roster.csv')

# roster_df.info()



In [149]:
### calculate the average age of each team
# roster_df.info()


tourney_roster = roster_df.copy()
full_tourney_roster = tourney_roster.copy()
 

# Calculate the average age for each team
# conver the DOB to datetime
tourney_roster['DOB'] = pd.to_datetime(tourney_roster['DOB'])
# Make new column for age in days
tourney_roster['Age'] = (pd.Timestamp.today() - tourney_roster['DOB']).dt.days / 365.25
average_age = tourney_roster.groupby('Team')['Age'].mean().reset_index()

# sort the teams by average age
average_age = average_age.sort_values('Age', ascending=True)



In [150]:
## State and Province Abbreviation library
canada_provinces = {'Ont.':'Ontario', 'B.C.':'British Columbia', 'Alb.':'Alberta', 'Que.':'Quebec', 'Man.':'Manitoba', 'Sask.': 'Saskatchewan', 'MB':'Manitoba', 'Sask':'Saskatchewan', 'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador',
                    'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador', 'YT.':'Yukon', 'N.W.T.':'Northwest Territories',
                    'Nun.':'Nunavut', 'NWT.':'Northwest Territories'}

us_states = {'Ala.':'Alabama', 'Alaska':'Alaska', 'Ariz.':'Arizona', 'Ark.':'Arkansas', 'Calif.':'California', 'Colo.':'Colorado', 'Conn.':'Connecticut', 'Del.':'Delaware', 'Fla.':'Florida',
                'Ga.':'Georgia', 'Hawaii':'Hawaii', 'Idaho':'Idaho', 'Ill.':'Illinois', 'Ind.':'Indiana', 'Iowa':'Iowa', 'Kan.':'Kansas', 'Ky.':'Kentucky', 'La.':'Louisiana', 'Maine':'Maine',
                'Md.':'Maryland', 'Mass.':'Massachusetts', 'Mich.':'Michigan', 'Minn.':'Minnesota', 'Miss.':'Mississippi', 'Mo.':'Missouri', 'Mont.':'Montana', 'Neb.':'Nebraska', 'Nev.':'Nevada',
                'N.H.':'New Hampshire', 'N.J.':'New Jersey', 'N.M.':'New Mexico', 'N.Y.':'New York', 'N.C.':'North Carolina', 'N.D.':'North Dakota', 'Ohio':'Ohio', 'Okla.':'Oklahoma', 'Ore.':'Oregon',
                'Pa.':'Pennsylvania', 'R.I.':'Rhode Island', 'S.C.':'South Carolina', 'S.D.':'South Dakota', 'Tenn.':'Tennessee', 'Tx':'Texas', 'Tex.':'Texas', 'Tex':'Texas', 'Utah':'Utah', 'Vt.':'Vermont', 'Va.':'Virginia',
                'Wash.':'Washington', 'W.Va.':'West Virginia', 'Wis.':'Wisconsin', 'Wisc.':'Wisconsin', 'Wyo.':'Wyoming'}
             

In [151]:
full_tourney_roster.head()

# Drop columns that are not needed - past the 7th column
full_tourney_roster = full_tourney_roster.drop(full_tourney_roster.columns[10:], axis=1)


full_tourney_roster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Team           1757 non-null   object 
 1   Player         1757 non-null   object 
 2   No             1757 non-null   float64
 3   Position       1757 non-null   object 
 4   Yr             1757 non-null   object 
 5   Ht             1756 non-null   object 
 6   Wt             1757 non-null   float64
 7   DOB            1757 non-null   object 
 8   Hometown       1755 non-null   object 
 9   Height_Inches  1757 non-null   float64
dtypes: float64(3), object(7)
memory usage: 137.4+ KB


In [152]:
#

# Split the Hometown column at the comma
full_tourney_roster[['Loc1', 'Loc2']] = full_tourney_roster['Hometown'].str.split(', ', expand=True)

# If the Loc2 column is is 'Alaska' add a period to the end of the Loc2 column
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x == 'Alaska' else x)
# Same with Ohio, Iowa, Maine
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['Ohio', 'Iowa', 'Maine', 'Utah', 'Texas'] else x)

# Same for canada provinces with NB, NWT and YT
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['NB', 'NWT', 'YT'] else x)

# If Loc2 does not have a . it is a country and should be moved to a new Country Column, Loc2 COlumn should be cleared
full_tourney_roster['Country'] = full_tourney_roster['Loc2'].apply(lambda x: x if isinstance(x, str) and '.' not in x else None)

# replace Loc2 columns without '.' with an empty string
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: '' if isinstance(x, str) and '.' not in x else x)



# Country Value Count
full_tourney_roster['Country'].value_counts()

# Apply the substitution dictionary to the Loc2 column - add countries to the Country column while doing the substitution
def assign_country_and_loc(row):
    # Check if the Loc2 value is in the canada_provinces dictionary
    if row['Loc2'] in canada_provinces:
        # Replace the Loc2 value
        row['Loc2'] = canada_provinces[row['Loc2']]
        # Assign 'Canada' to the Country column
        row['Country'] = 'Canada'
    # Check if the Loc2 value is in the us_states dictionary
    elif row['Loc2'] in us_states:
        # Replace the Loc2 value
        row['Loc2'] = us_states[row['Loc2']]
        # Assign 'USA' to the Country column
        row['Country'] = 'USA'
    # If the Loc2 value is not in either dictionary, do not change the Country value
    return row

# Strip the City, State, and COuntry column of any leading or trailing whitespace
full_tourney_roster['Loc1'] = full_tourney_roster['Loc1'].str.strip()
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].str.strip()


# Apply the function to each row
full_tourney_roster = full_tourney_roster.apply(assign_country_and_loc, axis=1)



# full_tourney_roster.sample(20)

full_tourney_roster['Loc2'].value_counts()
# full_tourney_roster['Country'].value_counts()
location_df = full_tourney_roster[['Team', 'Player', 'Position', 'Loc1', 'Loc2', 'Country']]
# Rename Loc1 and Loc2 to City and State/Province
location_df = location_df.rename(columns={'Loc1': 'City', 'Loc2': 'State/Province'})
# Strip the City, State, and COuntry column of any leading or trailing whitespace
location_df['City'] = location_df['City'].str.strip()
location_df['State/Province'] = location_df['State/Province'].str.strip()
location_df['Country'] = location_df['Country'].str.strip()

# If the State/Province is Alaska. set it to Alaska
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Alaska' if x == 'Alaska.' else x)
# Add USA to the Country column for Alaska
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Alaska' else x['Country'], axis=1)
# Same for Iowa and Ohio and Utah and Texas
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Texas' if x == 'Texas.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Texas' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Utah' if x == 'Utah.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Utah' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Idaho' if x == 'Idaho.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Idaho' else x['Country'], axis=1)

location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Iowa' if x == 'Iowa.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Iowa' else x['Country'], axis=1)
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Ohio' if x == 'Ohio.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Ohio' else x['Country'], axis=1)
# Same for Maine
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Maine' if x == 'Maine.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Maine' else x['Country'], axis=1)




# location_df.head(20)

In [153]:
location_df['State/Province'].value_counts()

# Country Value Count
location_df['Country'].value_counts()

# Show the Tyrone Bronte example
location_df[location_df['Player'] == 'Tyrone Bronte']

location_df[location_df['Country'] == 'Yukon']


,Team,Player,Position,City,State/Province,Country
325,RIT,Simon Isabelle,Forwards,Whitehorse,,Yukon


In [154]:
## HOT FIX for the weird ones
# If Country = Tx fix the row - State/Province = Texas, Country = USA
location_df.loc[location_df['Country'] == 'Tx', 'State/Province'] = 'Texas'
location_df.loc[location_df['Country'] == 'Tx', 'Country'] = 'USA'



# Same THing for MB (Manatoba)
location_df.loc[location_df['Country'] == 'MB', 'State/Province'] = 'Manitoba'
location_df.loc[location_df['Country'] == 'MB', 'Country'] = 'Canada'

# Same THing for Yukon 
location_df.loc[location_df['Country'] == 'Yukon', 'State/Province'] = 'Yukon'
location_df.loc[location_df['State/Province'] == 'Yukon', 'Country'] = 'Canada'


# IF Country = AUT fix the row - Country = Austria
location_df.loc[location_df['Country'] == 'AUT', 'Country'] = 'Austria'

# Fix MTech player incorrectly from Austria, Change to Australia (Tyrone Bronte)
location_df.loc[location_df['Player'] == 'Tyrone Bronte', 'Country'] = 'Australia'


location_df['Country'].value_counts()


# Export the location data to a CSV file
location_df.to_csv(f'{TEMP_FOLDER}Overall_player_hometown_table.csv', index=False)





In [155]:
# Get a count of the # of US States, Canadian Provinces, and Countries represented in the tournament
us_states_count = location_df[location_df['Country'] == 'USA']['State/Province'].nunique()
canadian_provinces_count = location_df[location_df['Country'] == 'Canada']['State/Province'].nunique()
countries_count = location_df['Country'].nunique()

us_states_count, canadian_provinces_count, countries_count

(40, 12, 22)

## Census Data

In [156]:
### Open Census 2023 Estimate Table

# Load the Census 2023 Estimate Table from DATA_FOLDER
census_df = pd.read_csv(f'{DATA_FOLDER}2023_Census_est.csv')

# Rename columns for easier access
census_df = census_df.rename(columns={'State or territory': 'State/Province', 'Census population[8][a] July 1, 2023 (est.)': 'Population_2023'})

# Display the first few rows of the Census DataFrame
# census_df.head()


In [157]:
location_df

state_df = location_df.groupby('State/Province').size().reset_index(name='Num_Players') # OLD
# state_df = location_df

# state_df

In [158]:
import pandas as pd



# Load US area data
us_area_df = pd.read_csv(f'{DATA_FOLDER}US_area.csv')

# Rename Land area km2 to Area_km2
us_area_df = us_area_df.rename(columns={'Land area km2': 'Area_km2', 'State / territory': 'State/Province'})

## Load Canadian Provinces Population Data
canada_pop_df = pd.read_csv(f'{DATA_FOLDER}Canada_2021_Census.csv')

# Rename columns for easier access
canada_pop_df = canada_pop_df.rename(columns={'Name': 'State/Province', 'Land area (km2)':'Area_km2'})

                                     
# Display the first few rows of the Canadian Population DataFrame
# canada_pop_df.head()

# Merge the state_df with the census_df on the 'State/Province' column
merged_df = state_df.merge(census_df, left_on='State/Province', right_on='State/Province', how='left')
merged_df = merged_df.merge(canada_pop_df, left_on='State/Province', right_on='State/Province', how='left')
# Merge Area
merged_df = merged_df.merge(us_area_df, left_on='State/Province', right_on='State/Province', how='left')
# Combine Area_km2 (USA) and Area_km2 (CAN) columns into a single column
merged_df['Area_km2'] = merged_df['Area_km2_x'].fillna(merged_df['Area_km2_y'])
# Drop old columns
merged_df = merged_df.drop(columns=['Area_km2_x', 'Area_km2_y', 'Total area km2'])

# Combine the Population_2023 (USA) and 2021_census (CAN) columns into a single column
merged_df['Population'] = merged_df['Population_2023'].fillna(merged_df['2021_census'])

# Drop columns that are no longer needed (2-4)
merged_df = merged_df.drop(columns=['Population_2023', '2021_census', '2021 census %'])

# Drop any rows with missing values in the Population column
merged_df = merged_df.dropna(subset=['Population'])
# remove all commas from the population column
merged_df['Population'] = merged_df['Population'].str.replace(',', '')
# Ensure the column is of type int
merged_df['Population'] = merged_df['Population'].astype(int)

# Calculate the number of players per 100,000 residents
merged_df['Players_per_100k'] = (merged_df['Num_Players'] / merged_df['Population']) * 100000

# Remove all , from the Area_km2 column then convert to float
merged_df['Area_km2'] = merged_df['Area_km2'].str.replace(',', '')
merged_df['Area_km2'] = merged_df['Area_km2'].astype(float)

# Calculate the number of players per 100 km^2
merged_df['Players_per_10k_km2'] = (merged_df['Num_Players'] / merged_df['Area_km2']) * 10000

# Save csv for checking
merged_df.to_csv(f'{TEMP_FOLDER}Overall_Players_per_capita.csv', index=False)

# merged_df


## END OVERALL

## START NCAA 

In [159]:
# Load roster data
roster_df = pd.read_csv(f'{DATA_FOLDER}roster.csv')

In [160]:

# Team Color Information
            # path to table with hex codes
team_color_path = os.path.join(DATA_FOLDER, 'team_color_book.csv')
team_colors = pd.read_csv(team_color_path)

team_colors

# Convert the 'Team' and 'Hex1' columns of team_colors_df into a dictionary
team_colors_dict = pd.Series(team_colors.Hex1.values, index=team_colors.Team).to_dict()

### calculate the average age of each team
# roster_df.info()

# filter to only the teams in the tournament (from team_colors_dict)
tourney_teams = list(team_colors_dict.keys())

tourney_roster = roster_df[roster_df['Team'].isin(tourney_teams)]
full_tourney_roster = tourney_roster.copy()
 


In [161]:
## State and Province Abbreviation library
canada_provinces = {'Ont.':'Ontario', 'B.C.':'British Columbia', 'Alb.':'Alberta', 'Que.':'Quebec', 'Man.':'Manitoba', 'Sask.': 'Saskatchewan', 'MB':'Manitoba', 'Sask':'Saskatchewan', 'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador',
                    'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador', 'YT.':'Yukon', 'N.W.T.':'Northwest Territories',
                    'Nun.':'Nunavut', 'NWT.':'Northwest Territories'}

us_states = {'Ala.':'Alabama', 'Alaska':'Alaska', 'Ariz.':'Arizona', 'Ark.':'Arkansas', 'Calif.':'California', 'Colo.':'Colorado', 'Conn.':'Connecticut', 'Del.':'Delaware', 'Fla.':'Florida',
                'Ga.':'Georgia', 'Hawaii':'Hawaii', 'Idaho':'Idaho', 'Ill.':'Illinois', 'Ind.':'Indiana', 'Iowa':'Iowa', 'Kan.':'Kansas', 'Ky.':'Kentucky', 'La.':'Louisiana', 'Maine':'Maine',
                'Md.':'Maryland', 'Mass.':'Massachusetts', 'Mich.':'Michigan', 'Minn.':'Minnesota', 'Miss.':'Mississippi', 'Mo.':'Missouri', 'Mont.':'Montana', 'Neb.':'Nebraska', 'Nev.':'Nevada',
                'N.H.':'New Hampshire', 'N.J.':'New Jersey', 'N.M.':'New Mexico', 'N.Y.':'New York', 'N.C.':'North Carolina', 'N.D.':'North Dakota', 'Ohio':'Ohio', 'Okla.':'Oklahoma', 'Ore.':'Oregon',
                'Pa.':'Pennsylvania', 'R.I.':'Rhode Island', 'S.C.':'South Carolina', 'S.D.':'South Dakota', 'Tenn.':'Tennessee', 'Tx':'Texas', 'Tex.':'Texas', 'Tex':'Texas', 'Utah':'Utah', 'Vt.':'Vermont', 'Va.':'Virginia',
                'Wash.':'Washington', 'W.Va.':'West Virginia', 'Wis.':'Wisconsin', 'Wisc.':'Wisconsin', 'Wyo.':'Wyoming'}
             

In [162]:
full_tourney_roster.head()

# Drop columns that are not needed - past the 7th column
full_tourney_roster = full_tourney_roster.drop(full_tourney_roster.columns[10:], axis=1)


full_tourney_roster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 438 entries, 112 to 1614
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Team           438 non-null    object 
 1   Player         438 non-null    object 
 2   No             438 non-null    float64
 3   Position       438 non-null    object 
 4   Yr             438 non-null    object 
 5   Ht             438 non-null    object 
 6   Wt             438 non-null    float64
 7   DOB            438 non-null    object 
 8   Hometown       438 non-null    object 
 9   Height_Inches  438 non-null    float64
dtypes: float64(3), object(7)
memory usage: 37.6+ KB


In [163]:
#

# Split the Hometown column at the comma
full_tourney_roster[['Loc1', 'Loc2']] = full_tourney_roster['Hometown'].str.split(', ', expand=True)

# If the Loc2 column is is 'Alaska' add a period to the end of the Loc2 column
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x == 'Alaska' else x)
# Same with Ohio, Iowa, Maine
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['Ohio', 'Iowa', 'Maine', 'Utah', 'Texas'] else x)

# Same for canada provinces with NB, NWT and YT
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['NB', 'NWT', 'YT'] else x)

# If Loc2 does not have a . it is a country and should be moved to a new Country Column, Loc2 COlumn should be cleared
full_tourney_roster['Country'] = full_tourney_roster['Loc2'].apply(lambda x: x if isinstance(x, str) and '.' not in x else None)

# replace Loc2 columns without '.' with an empty string
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: '' if isinstance(x, str) and '.' not in x else x)



# Country Value Count
full_tourney_roster['Country'].value_counts()

# Apply the substitution dictionary to the Loc2 column - add countries to the Country column while doing the substitution
def assign_country_and_loc(row):
    # Check if the Loc2 value is in the canada_provinces dictionary
    if row['Loc2'] in canada_provinces:
        # Replace the Loc2 value
        row['Loc2'] = canada_provinces[row['Loc2']]
        # Assign 'Canada' to the Country column
        row['Country'] = 'Canada'
    # Check if the Loc2 value is in the us_states dictionary
    elif row['Loc2'] in us_states:
        # Replace the Loc2 value
        row['Loc2'] = us_states[row['Loc2']]
        # Assign 'USA' to the Country column
        row['Country'] = 'USA'
    # If the Loc2 value is not in either dictionary, do not change the Country value
    return row

# Strip the City, State, and COuntry column of any leading or trailing whitespace
full_tourney_roster['Loc1'] = full_tourney_roster['Loc1'].str.strip()
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].str.strip()


# Apply the function to each row
full_tourney_roster = full_tourney_roster.apply(assign_country_and_loc, axis=1)



# full_tourney_roster.sample(20)

full_tourney_roster['Loc2'].value_counts()
# full_tourney_roster['Country'].value_counts()
location_df = full_tourney_roster[['Team', 'Player', 'Position', 'Loc1', 'Loc2', 'Country']]
# Rename Loc1 and Loc2 to City and State/Province
location_df = location_df.rename(columns={'Loc1': 'City', 'Loc2': 'State/Province'})
# Strip the City, State, and COuntry column of any leading or trailing whitespace
location_df['City'] = location_df['City'].str.strip()
location_df['State/Province'] = location_df['State/Province'].str.strip()
location_df['Country'] = location_df['Country'].str.strip()

# If the State/Province is Alaska. set it to Alaska
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Alaska' if x == 'Alaska.' else x)
# Add USA to the Country column for Alaska
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Alaska' else x['Country'], axis=1)
# Same for Iowa and Ohio and Utah and Texas
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Texas' if x == 'Texas.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Texas' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Utah' if x == 'Utah.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Utah' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Idaho' if x == 'Idaho.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Idaho' else x['Country'], axis=1)

location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Iowa' if x == 'Iowa.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Iowa' else x['Country'], axis=1)
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Ohio' if x == 'Ohio.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Ohio' else x['Country'], axis=1)
# Same for Maine
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Maine' if x == 'Maine.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Maine' else x['Country'], axis=1)




# location_df.head(20)

In [164]:
## HOT FIX for the weird ones
# If Country = Tx fix the row - State/Province = Texas, Country = USA
location_df.loc[location_df['Country'] == 'Tx', 'State/Province'] = 'Texas'
location_df.loc[location_df['Country'] == 'Tx', 'Country'] = 'USA'



# Same THing for MB (Manatoba)
location_df.loc[location_df['Country'] == 'MB', 'State/Province'] = 'Manitoba'
location_df.loc[location_df['Country'] == 'MB', 'Country'] = 'Canada'

# Same THing for Yukon 
location_df.loc[location_df['Country'] == 'Yukon', 'State/Province'] = 'Yukon'
location_df.loc[location_df['State/Province'] == 'Yukon', 'Country'] = 'Canada'


# IF Country = AUT fix the row - Country = Austria
location_df.loc[location_df['Country'] == 'AUT', 'Country'] = 'Austria'

# Fix MTech player incorrectly from Austria, Change to Australia (Tyrone Bronte)
location_df.loc[location_df['Player'] == 'Tyrone Bronte', 'Country'] = 'Australia'


location_df['Country'].value_counts()


# Export the location data to a CSV file
location_df.to_csv(f'{TEMP_FOLDER}NCAA_player_hometown_table.csv', index=False)





In [165]:
# Get a count of the # of US States, Canadian Provinces, and Countries represented in the tournament
us_states_count = location_df[location_df['Country'] == 'USA']['State/Province'].nunique()
canadian_provinces_count = location_df[location_df['Country'] == 'Canada']['State/Province'].nunique()
countries_count = location_df['Country'].nunique()

us_states_count, canadian_provinces_count, countries_count

(29, 11, 13)

In [166]:
# Get a count of the # of US States, Canadian Provinces, and Countries represented in the tournament
us_states_count = location_df[location_df['Country'] == 'USA']['State/Province'].nunique()
canadian_provinces_count = location_df[location_df['Country'] == 'Canada']['State/Province'].nunique()
countries_count = location_df['Country'].nunique()

us_states_count, canadian_provinces_count, countries_count

(29, 11, 13)

In [167]:
location_df

state_df = location_df.groupby('State/Province').size().reset_index(name='Num_Players') # OLD
# state_df = location_df

# state_df

In [168]:
import pandas as pd



# Load US area data
us_area_df = pd.read_csv(f'{DATA_FOLDER}US_area.csv')

# Rename Land area km2 to Area_km2
us_area_df = us_area_df.rename(columns={'Land area km2': 'Area_km2', 'State / territory': 'State/Province'})

## Load Canadian Provinces Population Data
canada_pop_df = pd.read_csv(f'{DATA_FOLDER}Canada_2021_Census.csv')

# Rename columns for easier access
canada_pop_df = canada_pop_df.rename(columns={'Name': 'State/Province', 'Land area (km2)':'Area_km2'})

                                     
# Display the first few rows of the Canadian Population DataFrame
# canada_pop_df.head()

# Merge the state_df with the census_df on the 'State/Province' column
merged_df = state_df.merge(census_df, left_on='State/Province', right_on='State/Province', how='left')
merged_df = merged_df.merge(canada_pop_df, left_on='State/Province', right_on='State/Province', how='left')
# Merge Area
merged_df = merged_df.merge(us_area_df, left_on='State/Province', right_on='State/Province', how='left')
# Combine Area_km2 (USA) and Area_km2 (CAN) columns into a single column
merged_df['Area_km2'] = merged_df['Area_km2_x'].fillna(merged_df['Area_km2_y'])
# Drop old columns
merged_df = merged_df.drop(columns=['Area_km2_x', 'Area_km2_y', 'Total area km2'])

# Combine the Population_2023 (USA) and 2021_census (CAN) columns into a single column
merged_df['Population'] = merged_df['Population_2023'].fillna(merged_df['2021_census'])

# Drop columns that are no longer needed (2-4)
merged_df = merged_df.drop(columns=['Population_2023', '2021_census', '2021 census %'])

# Drop any rows with missing values in the Population column
merged_df = merged_df.dropna(subset=['Population'])
# remove all commas from the population column
merged_df['Population'] = merged_df['Population'].str.replace(',', '')
# Ensure the column is of type int
merged_df['Population'] = merged_df['Population'].astype(int)

# Calculate the number of players per 100,000 residents
merged_df['Players_per_100k'] = (merged_df['Num_Players'] / merged_df['Population']) * 100000

# Remove all , from the Area_km2 column then convert to float
merged_df['Area_km2'] = merged_df['Area_km2'].str.replace(',', '')
merged_df['Area_km2'] = merged_df['Area_km2'].astype(float)

# Calculate the number of players per 100 km^2
merged_df['Players_per_10k_km2'] = (merged_df['Num_Players'] / merged_df['Area_km2']) * 10000

# Save csv for checking
merged_df.to_csv(f'{TEMP_FOLDER}NCAA_Players_per_capita.csv', index=False)

# merged_df


## END NCAA 

## COMBINE TO SINGLE EXCEL BOOK

In [169]:
## Combine 4 csv files into one excel file, each csv a different sheet
# Load the 4 CSV files
directory = '../data/tableau_tables/'



In [171]:
import pandas as pd
import os



# The path for the Excel file you want to create
excel_file_path = f'{directory}Combined_Book.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Iterate over each file in the directory
    for filename in os.listdir(TEMP_FOLDER):
        if filename.endswith('.csv'):
            # Construct the full file path
            file_path = os.path.join(TEMP_FOLDER, filename)
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(file_path)
            
            # Convert the filename to a valid Excel sheet name
            sheet_name = filename.replace('.csv', '')[:31]
            
            # Write the DataFrame to a new sheet in the Excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f'All CSV files have been combined into {excel_file_path}')


All CSV files have been combined into ../data/tableau_tables/Combined_Book.xlsx
